## COD API, Matches --> explore, format, clean & reshape

#### import stuff to work with

In [1]:
import asyncio
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

In [2]:
# Using SSO
# We're storing our SSO token in an .env file stored locally to separate our config from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

In [3]:
# we're storing our strings conversion module above current working (notebooks) directory
sys.path.insert(0, os.path.abspath('../wzkd'))
import labels
from labels import MODES_LABELS

In [41]:
# This time we're adding additional methods in the Call of Duty .py client only as there is no need to modify the HTTP class that already contains the endpoint we want to use
# Import the Class we want to modify

import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request


# following additional methods to be added in callofduty.client.py Client Class
# see notebooks/cod_api_doc.ipnyb for details

async def GetMatches(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # API res was filtered out here

    return data


async def GetMatchesDetailed(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['matches'] # API res was filtered out here


async def GetMatchesSummary(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['summary'] # API res was filtered out here


# add our modified methods into callofduty Client Class

Client.GetMatches = GetMatches
Client.GetMatchesDetailed = GetMatchesDetailed
Client.GetMatchesSummary = GetMatchesSummary

# raw_matches_datailed has indeed two keys : summary and matches (the one that is filtered out in the Client)
matches = await client.GetMatchesDetailed("psn", "confetti_seeker", Title.ModernWarfare, Mode.Warzone, limit=20) #AMADEVS#1689


# option save your result if you want to work in offline mode
with open("matches.pkl", 'wb') as f:
    pickle.dump(matches, f)

### Overview of returned stats

In [42]:
# option : load previously saved offline data i
with open('matches.pkl', 'rb') as f:
    loaded_matches = pickle.load(f)

In [43]:
pprint(loaded_matches[0], depth=2)

{'draw': False,
 'duration': 1614000,
 'gameType': 'wz',
 'map': 'mp_don4',
 'matchID': '16569868366653777065',
 'mode': 'br_brsolo',
 'player': {'awards': {},
            'brMissionStats': {...},
            'loadout': [...],
            'loadouts': [...],
            'rank': 54.0,
            'team': 'team_fifty_nine',
            'uno': '13831131263765265947',
            'username': 'Confetti_Seeker'},
 'playerCount': 151,
 'playerStats': {'assists': 0.0,
                 'bonusXp': 0.0,
                 'challengeXp': 0.0,
                 'damageDone': 285.0,
                 'damageTaken': 276.0,
                 'deaths': 2.0,
                 'distanceTraveled': 253666.6,
                 'executions': 0.0,
                 'gulagDeaths': 0.0,
                 'gulagKills': 1.0,
                 'headshots': 0.0,
                 'kdRatio': 0.5,
                 'kills': 1.0,
                 'longestStreak': 1.0,
                 'matchXp': 5666.0,
                 'medalXp':

#### raw data

In [44]:
raw = pd.DataFrame(loaded_matches)
raw.head(5)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1637585811,1637587425,mp_don4,br_brsolo,16569868366653777065,1614000,None,1,wz,151,"{'kills': 1.0, 'medalXp': 0.0, 'matchXp': 5666...","{'team': 'team_fifty_nine', 'rank': 54.0, 'awa...",150,None,False,False
1,1637584982,1637586572,mp_don4,br_brsolo,8714510796497931274,1590000,None,1,wz,150,"{'kills': 2.0, 'medalXp': 120.0, 'matchXp': 69...","{'team': 'team_twenty_nine', 'rank': 54.0, 'aw...",150,None,False,False
2,1637584205,1637585790,mp_don4,br_brsolo,11055215595461203138,1585000,None,1,wz,148,"{'kills': 4.0, 'medalXp': 20.0, 'objectiveTeam...","{'team': 'team_five', 'rank': 54.0, 'awards': ...",148,None,False,False
3,1637575541,1637577167,mp_don4,br_brsolo,5308570063444632311,1626000,None,1,wz,150,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 706....","{'team': 'team_eighty_three', 'rank': 54.0, 'a...",150,None,False,False
4,1637575311,1637576175,mp_escape3,br_rebirth_rbrthquad,6626896444167093817,864000,None,1,wz,41,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 0.0,...","{'team': 'team_five', 'rank': 54.0, 'awards': ...",12,None,False,False


In [45]:
raw.keys()

Index(['utcStartSeconds', 'utcEndSeconds', 'map', 'mode', 'matchID',
       'duration', 'playlistName', 'version', 'gameType', 'playerCount',
       'playerStats', 'player', 'teamCount', 'rankedTeams', 'draw',
       'privateMatch'],
      dtype='object')

#### playerStats col

In [46]:
player_stats = raw['playerStats'].apply(pd.Series)
player_stats.head(20)

,kills,medalXp,matchXp,scoreXp,wallBangs,score,totalXp,headshots,assists,challengeXp,...,objectiveTeamWiped,objectiveLastStandKill,objectiveBrDownEnemyCircle1,objectiveBrKioskBuy,objectiveBrDownEnemyCircle2,objectiveBrMissionPickupTablet,objectiveReviver,objectiveBrDownEnemyCircle6,objectiveMunitionsBoxTeammateUsed,objectiveBrDownEnemyCircle3
0,1.0,0.0,5666.0,1350.0,0.0,1350.0,7016.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,120.0,6977.0,1300.0,0.0,1000.0,8449.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,20.0,6848.0,2925.0,0.0,2925.0,9793.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,706.0,100.0,0.0,100.0,806.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,10.0,0.0,1850.0,0.0,1350.0,1887.0,1.0,0.0,0.0,...,NaN,2.0,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN
6,2.0,40.0,3172.0,1175.0,0.0,775.0,4387.0,0.0,0.0,0.0,...,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
7,0.0,0.0,4100.0,325.0,0.0,325.0,4430.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
8,1.0,0.0,9065.0,2195.0,0.0,1275.0,11297.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,NaN,NaN
9,1.0,0.0,1386.0,525.0,0.0,525.0,2186.0,0.0,1.0,0.0,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
player_stats.keys()

Index(['kills', 'medalXp', 'matchXp', 'scoreXp', 'wallBangs', 'score',
       'totalXp', 'headshots', 'assists', 'challengeXp', 'rank',
       'scorePerMinute', 'distanceTraveled', 'teamSurvivalTime', 'deaths',
       'kdRatio', 'bonusXp', 'gulagDeaths', 'timePlayed', 'executions',
       'gulagKills', 'nearmisses', 'objectiveBrCacheOpen', 'percentTimeMoving',
       'miscXp', 'longestStreak', 'teamPlacement', 'damageDone', 'damageTaken',
       'objectiveTeamWiped', 'objectiveLastStandKill',
       'objectiveBrDownEnemyCircle1', 'objectiveBrKioskBuy',
       'objectiveBrDownEnemyCircle2', 'objectiveBrMissionPickupTablet',
       'objectiveReviver', 'objectiveBrDownEnemyCircle6',
       'objectiveMunitionsBoxTeammateUsed', 'objectiveBrDownEnemyCircle3'],
      dtype='object')

In [48]:
player_stats[['headshots', 'distanceTraveled', 'teamSurvivalTime', 'objectiveBrKioskBuy']].head(5)

,headshots,distanceTraveled,teamSurvivalTime,objectiveBrKioskBuy
0,0.0,253666.600,423648.0,NaN
1,1.0,291781.470,596400.0,NaN
2,0.0,235086.060,601056.0,1.0
3,0.0,343678.100,106752.0,NaN
4,0.0,72389.445,71472.0,NaN


#### player col

In [49]:
player = raw['player'].apply(pd.Series)
player.head(5)

,team,rank,awards,username,uno,loadouts,brMissionStats,loadout
0,team_fifty_nine,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"{'missionsComplete': 0, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...
1,team_twenty_nine,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_ar_t9standard...,"{'missionsComplete': 0, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_ar_t9standard...
2,team_five,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"{'missionsComplete': 0, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...
3,team_eighty_three,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"{'missionsComplete': 0, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...
4,team_five,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_ar_t9standard...,"{'missionsComplete': 0, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_ar_t9standard...


In [50]:
player.keys()

Index(['team', 'rank', 'awards', 'username', 'uno', 'loadouts',
       'brMissionStats', 'loadout'],
      dtype='object')

#### Deep dive into player.loadout entry

In [51]:
# Each entry of 'loadout' is a list of dict. Either one dict (if 1 loadout) or more(2 or more loadouts taken)
player['loadout'][11][0]

{'primaryWeapon': {'name': 'iw8_sm_t9fastfire',
  'label': None,
  'imageLoot': None,
  'imageIcon': None,
  'variant': '0',
  'attachments': [{'name': 'stockcust',
    'label': None,
    'image': None,
    'category': None},
   {'name': 'gripvertpro', 'label': None, 'image': None, 'category': None},
   {'name': 'smags2', 'label': None, 'image': None, 'category': None},
   {'name': 'barcust2', 'label': None, 'image': None, 'category': None},
   {'name': 'silencer2', 'label': None, 'image': None, 'category': None}]},
 'secondaryWeapon': {'name': 'iw8_sn_t9accurate',
  'label': None,
  'imageLoot': None,
  'imageIcon': None,
  'variant': '0',
  'attachments': [{'name': 'laserrange',
    'label': None,
    'image': None,
    'category': None},
   {'name': 'pistolgrip01', 'label': None, 'image': None, 'category': None},
   {'name': 'gripang', 'label': None, 'image': None, 'category': None},
   {'name': 'barcust', 'label': None, 'image': None, 'category': None},
   {'name': 'silencer', 'lab

In [52]:
# second loadout structure 
if len(player['loadout'][11]) > 1:
    player['loadout'][11][1].keys()

In [53]:
player['loadout'][11][0]['primaryWeapon'].keys()

dict_keys(['name', 'label', 'imageLoot', 'imageIcon', 'variant', 'attachments'])

#### expanded data : raw + playerStats & player columns expanded + player.loadout expanded

In [54]:
expanded = pd.DataFrame(loaded_matches)
expanded = pd.concat([expanded.drop(['playerStats'], axis=1), expanded['playerStats'].apply(pd.Series)], axis=1)
expanded = pd.concat([expanded.drop(['player'], axis=1), expanded['player'].apply(pd.Series)], axis=1)
expanded = expanded.drop(['brMissionStats'], axis = 1)

# player.loadout column
expanded = pd.concat([expanded.drop(['loadout'], axis=1), expanded['loadout'].apply(pd.Series)], axis=1)
for col in range(0,3):
    if col in expanded.columns:
        expanded[col] = expanded[col].apply(lambda x: [x['primaryWeapon']['name'],x['secondaryWeapon']['name']] if not str(x) == 'nan' else np.nan)
        expanded = expanded.rename(columns={col: f"loadout_{str(col +1)}"})
expanded.head(15)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,...,team,rank,awards,username,uno,loadouts,loadout_1,loadout_2,loadout_3,3
0,1637585811,1637587425,mp_don4,br_brsolo,16569868366653777065,1614000,None,1,wz,151,...,team_fifty_nine,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"[iw8_sm_t9fastfire, iw8_sn_t9accurate]","[iw8_pi_t9semiauto, iw8_fists]",NaN,NaN
1,1637584982,1637586572,mp_don4,br_brsolo,8714510796497931274,1590000,None,1,wz,150,...,team_twenty_nine,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_ar_t9standard...,"[iw8_ar_t9standard, iw8_sm_t9fastfire]","[iw8_pi_t9semiauto, iw8_fists]",NaN,NaN
2,1637584205,1637585790,mp_don4,br_brsolo,11055215595461203138,1585000,None,1,wz,148,...,team_five,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"[iw8_sm_t9fastfire, iw8_sn_t9accurate]","[iw8_pi_t9semiauto, iw8_fists]",NaN,NaN
3,1637575541,1637577167,mp_don4,br_brsolo,5308570063444632311,1626000,None,1,wz,150,...,team_eighty_three,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"[iw8_sm_t9fastfire, iw8_sn_t9accurate]","[iw8_pi_t9semiauto, iw8_fists]",NaN,NaN
4,1637575311,1637576175,mp_escape3,br_rebirth_rbrthquad,6626896444167093817,864000,None,1,wz,41,...,team_five,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_ar_t9standard...,"[iw8_ar_t9standard, iw8_sm_t9fastfire]","[iw8_pi_papa320, iw8_fists]",NaN,NaN
5,1637574568,1637575365,mp_escape3,br_rebirth_rbrthquad,13523893993387186047,797000,None,1,wz,39,...,team_four,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"[iw8_sm_t9fastfire, iw8_sn_t9accurate]","[iw8_pi_papa320, iw8_fists]","[iw8_ar_t9standard, iw8_sm_t9fastfire]",NaN
6,1637531570,1637532923,mp_don4,br_vov_op_flash,11872838025544084186,1353000,None,1,wz,132,...,team_thirteen,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"[iw8_sm_t9fastfire, iw8_sn_t9accurate]","[iw8_pi_golf21, iw8_me_t9battleaxe]",NaN,NaN
7,1637530905,1637532255,mp_don4,br_vov_op_flash,7507402071651243197,1350000,None,1,wz,132,...,team_five,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"[iw8_sm_t9fastfire, iw8_sn_t9accurate]","[iw8_pi_golf21, iw8_me_t9battleaxe]",NaN,NaN
8,1637529617,1637530985,mp_don4,br_vov_op_flash,5640527527103566278,1368000,None,1,wz,131,...,team_thirteen,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"[iw8_sm_t9fastfire, iw8_sn_t9accurate]","[iw8_pi_golf21, iw8_me_t9etool]","[iw8_pi_t9revolver, iw8_fists]","{'primaryWeapon': {'name': 'iw8_pi_golf21', 'l..."
9,1637517515,1637519100,mp_don4,br_brduos,7778717125983808406,1585000,None,1,wz,151,...,team_forty_five,54.0,{},Confetti_Seeker,13831131263765265947,[{'primaryWeapon': {'name': 'iw8_sm_t9fastfire...,"[iw8_sm_t9fastfire, iw8_sn_t9accurate]","[iw8_pi_t9semiauto, iw8_fists]",NaN,NaN


#### Functions

In [55]:
def getLastMatchId(matches):
    """ Extract last (Battle Royale) Match ID from list of last matches """
    list_br_match = [match for match in matches if "br_br" in match["mode"]]
    last_match_id = int(list_br_match[0]["matchID"]) if len(list_br_match) > 0 else None
    return last_match_id


def getGamertag(matches):
    """ 
    A player can be searchable with a given name but having a different gamertag in-game
    We need that gamertag if we were to analyse the "one match stats" endpoint (as done in match_format.py)
    from player's perspective
    """
    return matches[0]['player']['username']

def MatchesToDf(matches):
    """
    Convert Matches API result to a DataFrame we we can perform our aggregations nicely, later.
    Expand some entries (i.e player, playerstats) that are deeply nested.
    Filter out / retains columns.
    
    Parameters
    ----------
    matches : result from COD API "matches" endpoint ; FYI formated as : 
        list[
                dict{ match 1 stats },
                dict{ match 2 stats },
                dict{ 20 matches max },
            ]
    
    Returns
    -------
    DataFrame, (max 20) matches as rows, matches' stats as columns/values
    """    
    
    
    keep_cols =  [
        'mode',
        'utcStartSeconds',
        'utcEndSeconds',
        'timePlayed',
        'teamPlacement',
        'kdRatio', 
        'kills', 
        'deaths', 
        'assists', 
        'damageDone',
        'damageTaken',
        'gulagKills',
        'percentTimeMoving',
        'duration',
        ]

    df = pd.DataFrame(matches)
    
    # column playerStats is a series of dict, we can expand it easily and append, then drop the original
    df = pd.concat([df.drop(['playerStats'], axis=1), df['playerStats'].apply(pd.Series)], axis=1)
    
    # colum player has more depth
    # once expanded, it has a column 'loadout' : a series of list of dict (either one or more, we will keep 1 to max 3)
    # and also brMissionStats (mostly empty ?, a col only present in BR matches) that we aren't interested in
    
    df = pd.concat([df.drop(['player'], axis=1), df['player'].apply(pd.Series)], axis=1)
    if 'brMissionStats' in df.columns:
        df = df.drop(['brMissionStats'], axis = 1)
    df = pd.concat([df.drop(['loadout'], axis=1), df['loadout'].apply(pd.Series)], axis=1)
    for col in range(0,3):
        if col in df.columns:
            df[col] = df[col].apply(lambda x: f"{x['primaryWeapon']['name']} - {x['secondaryWeapon']['name']}" if not str(x) == 'nan' else np.nan)
            col_name = "loadout_" + str(col +1)
            df = df.rename(columns={col: f"loadout_{str(col +1)}"})   
            keep_cols.append(col_name)
    
    return df[keep_cols]

In [56]:
df = MatchesToDf(loaded_matches)
df.head(3)

,mode,utcStartSeconds,utcEndSeconds,timePlayed,teamPlacement,kdRatio,kills,deaths,assists,damageDone,damageTaken,gulagKills,percentTimeMoving,duration,loadout_1,loadout_2,loadout_3
0,br_brsolo,1637585811,1637587425,484.0,67.0,0.5,1.0,2.0,0.0,285.0,276.0,1.0,91.54930,1614000,iw8_sm_t9fastfire - iw8_sn_t9accurate,iw8_pi_t9semiauto - iw8_fists,NaN
1,br_brsolo,1637584982,1637586572,656.0,56.0,1.0,2.0,2.0,0.0,382.0,330.0,1.0,87.24832,1590000,iw8_ar_t9standard - iw8_sm_t9fastfire,iw8_pi_t9semiauto - iw8_fists,NaN
2,br_brsolo,1637584205,1637585790,634.0,55.0,2.0,4.0,2.0,0.0,779.0,230.0,1.0,97.15243,1585000,iw8_sm_t9fastfire - iw8_sn_t9accurate,iw8_pi_t9semiauto - iw8_fists,NaN


In [65]:
last_match_id = getLastMatchId(loaded_matches)
last_match_id

16569868366653777065

In [21]:
def MatchesStandardize(df):
    """
    A first layer of standadization (as : properly formated) to our matches DataFrame
    For further aggregations / better readibility of our data
    
    Returns
    -------
    DataFrame : matches as rows, cleaned matches/player stats as columns
    """
    
    int_cols =  [
        'teamPlacement', 
        'kills', 
        'deaths', 
        'assists', 
        'gulagKills', 
        'damageDone',
        'damageTaken'
        ]
    
    float_cols = [
        'kdRatio',
        'percentTimeMoving'
        ]
    
    ts_cols = [
        'utcStartSeconds',
        'utcEndSeconds'
        ]
    
    columns_labels = {
        'utcEndSeconds':'Ended at',
        'utcStartSeconds':'Started at',
        'timePlayed': 'Playtime',
        'teamPlacement':'#',
        'kdRatio':'KD',
        'damageDone':'Damage >',
        'damageTaken':'Damage <',
        'gulagKills':'Gulag',
        'headshots':'% headshots',
        'percentTimeMoving':'% moving',
        'duration':'Game duration'
        }

    df = df.fillna(0) # sometimes (Plunder only ?) rank, gulag is NaN
    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float).round(1) # still renders 0.0000 in streamlit but ugly hacks exists
    
    # specials
    df['utcEndSeconds'] = df['utcEndSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    df['utcStartSeconds'] = df['utcStartSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    
    df['duration'] = df['duration'].apply(lambda x: x/1000).apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M')) # API duration is in seconds x1000
    df['timePlayed'] = df['timePlayed'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M:%S')) # API timePlayed is in seconds
    if 'gulagKills' in df.columns.tolist():
        df.loc[df['mode'].isin(MODES_LABELS['battle_royale']), 'gulagKills'] = df.loc[df['mode'].isin(MODES_LABELS['battle_royale']), 'gulagKills'].map({1:'W', 0:'L'})
        df['gulagKills'].replace(0, '-', inplace=True)
    for col in ['loadout_1', 'loadout_2', 'loadout_3']:
        df.fillna({col:'-'}, inplace=True) if col in df.columns else None
    
    for label_cat in ['battle_royale', 'multiplayer']:
        df = df.replace({"mode": MODES_LABELS[label_cat]})
    df = df.rename(columns=columns_labels)
    df.columns = df.columns.str.capitalize()
    df = df.rename({"Kd":"KD"}, axis=1)
     
    return df

In [22]:
raw = pd.DataFrame(loaded_matches)
df = MatchesToDf(raw)
df = MatchesStandardize(df)
df.head(8)

,Mode,Started at,Ended at,Playtime,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,% moving,Game duration,Loadout_1,Loadout_2,Loadout_3
0,Trios,2021-11-21 23:57:09,2021-11-22 00:23:52,27:55,6,1.0,2,2,1,814,301,L,71.4,26,iw8_ar_valpha - iw8_sn_t9accurate,iw8_pi_t9semiauto - iw8_fists,0
1,Trios,2021-11-21 23:45:57,2021-11-22 00:11:49,10:13,35,0.5,1,2,0,146,530,W,93.3,25,iw8_sm_t9standard - iw8_sn_t9accurate,0,0
2,Trios,2021-11-21 23:15:46,2021-11-21 23:43:18,29:02,1,6.0,6,0,2,3100,469,L,98.8,27,iw8_ar_valpha - iw8_sn_t9accurate,iw8_pi_t9semiauto - iw8_fists,0
3,Trios,2021-11-21 23:04:21,2021-11-21 23:30:21,10:11,39,1.0,2,2,0,490,476,W,82.9,26,iw8_sm_t9standard - iw8_sn_t9accurate,0,0
4,S6 Flashback,2021-11-21 22:52:50,2021-11-21 23:15:23,09:17,28,0.0,0,2,0,174,384,L,80.5,22,iw8_ar_valpha - iw8_sn_t9accurate,iw8_pi_golf21 - iw8_me_t9battleaxe,0
5,S6 Flashback,2021-11-21 22:41:45,2021-11-21 23:04:15,09:02,26,0.5,1,2,0,579,377,W,99.1,22,iw8_ar_valpha - iw8_sn_t9accurate,iw8_pi_golf21 - iw8_me_t9battleaxe,0
6,S6 Flashback,2021-11-21 22:20:17,2021-11-21 22:43:05,20:03,10,1.0,3,3,2,1121,620,W,78.2,22,iw8_ar_valpha - iw8_sn_t9accurate,iw8_pi_golf21 - iw8_me_t9etool,iw8_pi_golf21 - iw8_me_t9etool
7,Duos,2021-11-21 22:06:24,2021-11-21 22:33:58,12:01,37,1.0,2,2,0,761,682,W,95.8,27,iw8_sm_t9standard - iw8_sn_t9accurate,0,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Mode           20 non-null     object        
 1   Started at     20 non-null     datetime64[ns]
 2   Ended at       20 non-null     datetime64[ns]
 3   Playtime       20 non-null     object        
 4   #              20 non-null     int64         
 5   KD             20 non-null     float64       
 6   Kills          20 non-null     int64         
 7   Deaths         20 non-null     int64         
 8   Assists        20 non-null     int64         
 9   Damage >       20 non-null     int64         
 10  Damage <       20 non-null     int64         
 11  Gulag          20 non-null     object        
 12  % moving       20 non-null     float64       
 13  Game duration  20 non-null     object        
 14  Loadout_1      20 non-null     object        
 15  Loadout_2      20 non-nul

In [24]:
loadout_cols = df.columns[df.columns.str.startswith('Loadout')].tolist()
loadout_cols

['Loadout_1', 'Loadout_2', 'Loadout_3']

In [38]:
def MatchesPerDay(df):
    """
    Final layer applied to our list of matches with stats, to render them in our Streamlit App
    Streamlit/basic AgGrid does not render well (aka w. blank rows etc.) multi indexed df
    So we structure and display our data differently (a dictionary instead of a df), in a daily manner
    
    Returns
    -------
    Dictionary :
    {
        "str_weekday_1":df-of-matches-that-day,
        "str_weekday_2":df-of-matches-that-day,
        (...)
    }
    """
    
    drop_cols =  [
        'Started at',
        'Playtime',
        '% moving',
        'Game duration'
        ]
    
    keep_cols = [
        'End time',
        'Mode',
        '#',
        'KD',
        'Kills',
        'Deaths',
        'Assists',
        'Damage >',
        'Damage <',
        'Gulag'
    ]
    
    loadout_cols = df.columns[df.columns.str.startswith('Loadout')].tolist()
    df[loadout_cols] = df[loadout_cols].replace(0, '-') # else can't concat Loadouts cols
    
    # 1. --- initial formating : datetime, concat loadouts cols ---
    
    df = df.drop(drop_cols, axis = 1)
    df['End time'] = df['Ended at'].dt.time
    
    def concat_loadouts(df, columns):
        return pd.Series(map(' , '.join, df[columns].values.tolist()),index = df.index)
    
    df['Weapons'] = concat_loadouts(df, loadout_cols)
    keep_cols = [*keep_cols, *['Weapons']]
    
    # 2. --- build result {"day1": df-matches-that-day, "day2": df...} ---
    
    list_df = [g for n, g in df.groupby(pd.Grouper(key='Ended at',freq='D'))]
    list_df = [df for df in list_df if not df.empty]
    list_days = [df['Ended at'].tolist()[0].strftime('%A, %d/%m') for df in list_df]
    
    # make sure we display latest day first, then build dictionary
    for list_ in [list_days, list_df]:
        list_.reverse()
    day_matches = dict(zip(list_days, list_df))
    
    # 3. --- some more (re)formating ---
    
    # for some reason (me ? ^-^, Grouper => Series?) couldnt' modify df before building the result, must iterate again 
    for k, v in day_matches.items():
        day_matches[k] = day_matches[k][keep_cols]
      
    return day_matches


def AggStats(df):
    """
    We want to add aggregated stats for each day/df of matches we got from MatchesPerDay()
    Each daily aggregation will be rendered in our Streamlit App on top of each list of matches
    
    Returns
    -------
    Dictionary :
    {
    'Kills':total n kills that day,
    'Deaths': total deaths,
    'KD': kills/deaths,
    'Gulags': win % ,
    'Played': count matches -
    }
    """

    agg_func = {
        "Mode":"count",
        "Kills":"sum",
        "Deaths":"sum"
    }
    
    kd = (df.Kills.sum() / df.Deaths.sum()).round(2)
    gulagWinRatio = int((df.Gulag.str.count("W").sum() *100) / len(df))
    
    dict_ = df.agg(agg_func).to_dict()
    dict_.update({'KD': kd})
    dict_.update({'Gulags': gulagWinRatio})
    dict_['Played'] = dict_.pop('Mode')
    
    
    return dict_

In [39]:
raw = pd.DataFrame(loaded_matches)
df = MatchesToDf(raw)
df = MatchesStandardize(df)
daily = MatchesPerDay(df)

In [40]:
for day in daily.keys():
    print(day)
    day_stats = AggStats(daily[day])
    print(day_stats)
    display(daily[day])
    #print(daily[day]['Mode'].values)

Monday, 22/11
{'Kills': 3, 'Deaths': 4, 'KD': 0.75, 'Gulags': 50, 'Played': 2}


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
1,00:11:49,Trios,35,0.5,1,2,0,146,530,W,"iw8_sm_t9standard - iw8_sn_t9accurate , - , -"
0,00:23:52,Trios,6,1.0,2,2,1,814,301,L,"iw8_ar_valpha - iw8_sn_t9accurate , iw8_pi_t9s..."


Sunday, 21/11
{'Kills': 78, 'Deaths': 106, 'KD': 0.74, 'Gulags': 44, 'Played': 18}


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
19,16:39:14,Trios,15,0.0,0,2,3,1148,582,L,"iw8_ar_valpha - iw8_sn_t9accurate , - , -"
18,17:05:23,Duos,5,0.2,1,4,1,606,672,W,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
17,17:35:06,Duos,10,1.0,2,2,1,1352,592,W,"iw8_ar_valpha - iw8_sn_t9accurate , - , -"
16,18:02:57,Trios,1,0.8,3,4,6,599,773,W,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
15,18:32:07,S6 Flashback,18,1.3,4,3,2,2038,575,L,"iw8_sm_t9standard - iw8_sn_t9accurate , iw8_pi..."
14,18:51:04,S6 Flashback,25,2.0,4,2,0,1400,502,W,"iw8_sm_t9standard - iw8_sn_t9accurate , iw8_pi..."
13,19:59:09,Pldr x1,0,0.4,5,14,3,1268,1430,-,"iw8_sm_t9standard - iw8_sn_t9accurate , iw8_sm..."
12,20:11:50,Pldr x1,0,0.9,8,9,6,2650,1000,-,"iw8_sm_t9standard - iw8_sn_t9accurate , - , -"
11,20:27:36,Pldr x1,0,0.4,5,13,0,1531,1576,-,"iw8_sm_t9standard - iw8_sn_t9accurate , - , -"
10,20:45:18,Pldr x1,0,1.6,19,12,3,4501,1480,-,"iw8_sm_t9standard - iw8_sn_t9accurate , - , -"


In [37]:
for k, v in daily.items():
    print(k)
    print(AggStats(v))

Monday, 22
{'Kills': 3, 'Deaths': 4, 'KD': 0.75, 'Gulags': 50, 'Played': 2}
Sunday, 21
{'Kills': 78, 'Deaths': 106, 'KD': 0.74, 'Gulags': 44, 'Played': 18}


In [30]:
df = daily["Monday"].copy(deep=True)
df

,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
1,00:11:49,Trios,35,0.5,1,2,0,146,530,W,"iw8_sm_t9standard - iw8_sn_t9accurate , - , -"
0,00:23:52,Trios,6,1.0,2,2,1,814,301,L,"iw8_ar_valpha - iw8_sn_t9accurate , iw8_pi_t9s..."
